In [ ]:
from itertools import chain, combinations, product
import json

import numpy as np
from scipy.stats import binned_statistic_2d
from quickbin import bin2d
from quickbin.definitions import check_ops, Ops 
from quickbin.benchmarks.benchmark import benchmark

In [ ]:
all_ops = chain(
    ((o,) for o in Ops),
    combinations((Ops.count, Ops.sum, Ops.mean), 2),
    combinations((Ops.count, Ops.sum, Ops.mean, Ops.std), 3),
    ((Ops.count, Ops.sum, Ops.mean, Ops.std), (Ops.min, Ops.max))
)
all_ops = tuple(map(Ops, map(sum, all_ops)))
assert all(map(check_ops, all_ops))

base_sizes = np.array([1000, 2000, 10000, 20000, 50000])
sizes = np.hstack([base_sizes, base_sizes * 100, base_sizes * 10000])
bincounts = (10, 25, 50, 100, 250, 500, 1000, 1500)

recs = []

In [ ]:
# with open("benchmarks_1.json") as stream:
#     recs = json.load(stream)

In [ ]:
checkmem = False
min_memcheck_size = 1000 * 1000 ** 2
for size, n_bins, ops in product(sizes, bincounts, all_ops):
    # we can just sum the scipy runtimes for multiple ops.
    which = ("quickbin",) if ops.bit_count() > 1 else ("quickbin", "scipy")
    if size <= 50000:
        n_iter = 25
    elif size < 5000000:
        n_iter = 10
    else:
        n_iter = 2
    baserec = {
        'size': int(size),
        'n_bins': n_bins,
        'ops': ops.name,
        'n_iter': n_iter,
        'checkmem': checkmem
    }
    if any(
        all(baserec[k] == rec.get(k) for k in baserec.keys())
        for rec in recs
    ):
        continue
    # this is a weird situation that is not _entirely_ useless but
    # generally unlikely to occur in the wild.
    if n_bins ** 2 > size:
        continue
    if checkmem is True and (n_bins ** 2 + size * 3) * 8 < min_memcheck_size:
        continue
    print(f"\n----size {size}, n_bins {n_bins}, ops {ops.name}----\n")
    rec = benchmark(
        size=size,
        n_bins=n_bins,
        ops=ops,
        which=which,
        n_iter=n_iter,
        checkmem=checkmem,
        verbose=True
    ) | baserec | {'ops': ops.name}
    recs.append(rec)

In [ ]:
with open("benchmarks_1.json", "w") as stream:
    json.dump(recs, stream)